# CyMetric Testing Notebook

This notebook tests individual components of the LinkSample function for debugging and validation.

Steps:
1. Import necessary libraries and functions
2. Load configuration
3. Generate CY points
4. Prepare CY model
5. Compute geometry (Kähler form, holomorphic volume form)
6. Compute G2 forms

#...to dooooooo:\\
~ check holomorphic to real function\\
~ happy with hermitian_to_kahler_real function (just takes imag part basically and arranges)?\\
~ learn holomorphic complex number (as 2 real numbers) from cy3 coord input\\
~ 

#notes:
~ the cy metric is hermitian up to 1e-9, but when explicitly hermitise makes it to machine precision!



## 1. Import Libraries and Functions

In [ ]:
# Import standard libraries
import os
import sys
import yaml
import numpy as np
np.set_printoptions(linewidth=200)
import tensorflow as tf
import pickle
import pathlib

# Setup path for cymetric package (from scratch/ to github/)
_parent_dir = pathlib.Path(os.getcwd()).parent
_cymetric_dir = _parent_dir.parent / "cymetric"
if str(_parent_dir) not in sys.path:
    sys.path.insert(0, str(_parent_dir))
if str(_cymetric_dir) not in sys.path:
    sys.path.insert(0, str(_cymetric_dir))

# Create alias to fix cymetric internal imports
import cymetric
if hasattr(cymetric, 'cymetric'):
    sys.modules['cymetric'] = cymetric.cymetric

# Import geometry functions
from geometry.geometry import (
    hermitian_to_kahler_real, 
    holomorphic_volume_form_to_real, 

    compute_gG2,
    exterior_derivative
)
from geometry.patches import CoordChange_C5R10
from geometry.wedge import wedge
from geometry.compression import form_to_vec, vec_to_form, vec_to_metric

# Import cymetric functions
from cymetric.pointgen.pointgen import PointGenerator
from cymetric.models.tfhelper import prepare_tf_basis
from cymetric.models.tfmodels import PhiFSModel

print("✅ All imports successful!")

✅ All imports successful!


## 2. Load Configuration

Test the configuration loading step.

In [ ]:
# Set up paths (go to parent directory)
cymodel_name = ''  # Set to '_name' if using named model
parent_dir = os.path.dirname(os.getcwd())
dirname = os.path.join(parent_dir, 'models', 'cy_models', 'link_data')
config_path = os.path.join(parent_dir, f'models/cy_models/cy_model_config{cymodel_name}.yaml')
cymodel_path = os.path.join(parent_dir, f'models/cy_models/cy_metric_model{cymodel_name}.keras')

# Load configuration
with open(config_path, 'r') as f:
    config = yaml.unsafe_load(f)

print("Configuration loaded:")
print(f"  Monomials shape: {np.array(config['monomials']).shape}")
print(f"  Coefficients shape: {np.array(config['coefficients']).shape}")
print(f"  Kähler moduli: {config['kmoduli']}")
print(f"  Ambient dimension: {config['ambient']}")
print(f"  Neural network layers: {config['nlayer']}")
print(f"  Hidden units: {config['nHidden']}")
print(f"  Activation: {config['act']}")
print(f"  Input dimension: {config['n_in']}")
print(f"  Output dimension: {config['n_out']}")
print(f"  Alpha: {config['alpha']}")

Configuration loaded:
  Monomials shape: (5, 5)
  Coefficients shape: (5,)
  Kähler moduli: [1.]
  Ambient dimension: [4]
  Neural network layers: 3
  Hidden units: 64
  Activation: gelu
  Input dimension: 10
  Output dimension: 1
  Alpha: [1.0, 1.0, 1.0, 1.0, 1.0]


## 3. Generate CY Points

Test the point generation process on the Calabi-Yau manifold.

In [ ]:
# Set number of points to generate
n_pts = 100  # Small number for testing
target_patch = None  # Set to [one_idx, dropped_idx] to filter for specific patch, or None for all

print(f"Generating {n_pts} CY points...")

# Initialize point generator
pg = PointGenerator(
    config['monomials'], 
    config['coefficients'], 
    config['kmoduli'], 
    config['ambient']
)

# Generate dataset
kappa = pg.prepare_dataset(n_pts, dirname, val_split=0.) #...this makes the dataset and saves it as a file to import
pg.prepare_basis(dirname, kappa=kappa)

# Load generated data
data = np.load(os.path.join(dirname, 'dataset.npz'))
points = data['X_train']

print(f"  Generated points shape: {points.shape}")
print(f"  Points are in R^10 (real coordinates of C^5)")

Generating 100 CY points...


/Users/ed/miniforge3/envs/g2_ml/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/ed/miniforge3/envs/g2_ml/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
pointgen:INFO:Vol_k: 4.999999999999999, Vol_cy: 277.6133527745711.


  Generated points shape: (100, 10)
  Points are in R^10 (real coordinates of C^5)


In [ ]:
##testinggg
np.where(one_idxs == dropped_idxs)

(array([], dtype=int64),)

### Convert to Complex Coordinates and Identify Patches

In [16]:
# Convert to C^5 complex coordinates
cy_points_C5 = CoordChange_C5R10(points, inverse=True)

print(f"Complex coordinates shape: {cy_points_C5.shape}")
print(f"Sample point: {cy_points_C5[0]}")

# Identify patch indices
one_idxs = np.argmax(np.isclose(cy_points_C5, complex(1, 0)), axis=1)
dropped_idxs = pg._find_max_dQ_coords(cy_points_C5)

print(f"\nPatch information:")
print(f"  one_idxs shape: {one_idxs.shape}")
print(f"  dropped_idxs shape: {dropped_idxs.shape}")
print(f"  Unique one_idxs: {np.unique(one_idxs)}")
print(f"  Unique dropped_idxs: {np.unique(dropped_idxs)}")

# Count patch distribution
unique_patches, patch_counts = np.unique(
    np.stack([one_idxs, dropped_idxs], axis=1), 
    axis=0, 
    return_counts=True
)
print(f"\nPatch distribution ({len(unique_patches)} unique patches):")
for i, (patch, count) in enumerate(zip(unique_patches, patch_counts)):
    print(f"  Patch {patch}: {count} points ({count/n_pts*100:.1f}%)")
    if i >= 9:  # Limit output
        print(f"  ... and {len(unique_patches) - 10} more patches")
        break

Complex coordinates shape: (100, 5)
Sample point: [-0.26571881-9.17297455e-01j -0.36345359+2.42049070e-01j
  1.        -1.92084502e-17j -0.59842131-1.43934436e-01j
 -0.35605605-6.91997143e-01j]

Patch information:
  one_idxs shape: (100,)
  dropped_idxs shape: (100,)
  Unique one_idxs: [0 1 2 3 4]
  Unique dropped_idxs: [0 1 2 3 4]

Patch distribution (20 unique patches):
  Patch [0 1]: 4 points (4.0%)
  Patch [0 2]: 6 points (6.0%)
  Patch [0 3]: 7 points (7.0%)
  Patch [0 4]: 7 points (7.0%)
  Patch [1 0]: 1 points (1.0%)
  Patch [1 2]: 7 points (7.0%)
  Patch [1 3]: 7 points (7.0%)
  Patch [1 4]: 2 points (2.0%)
  Patch [2 0]: 4 points (4.0%)
  Patch [2 1]: 5 points (5.0%)
  ... and 10 more patches


### Generate Link Points in Local R^7 Coordinates

In [17]:
# Generate random angles for the link
thetas = np.random.uniform(low=0., high=2*np.pi, size=cy_points_C5.shape[0])

# Create mask to extract C^3 coordinates (excluding one_idx and dropped_idx)
mask = np.ones(cy_points_C5.shape, dtype=bool)
samples = np.arange(cy_points_C5.shape[0])
mask[samples, one_idxs] = False
mask[samples, dropped_idxs] = False

# Extract C^3 coordinates and convert to R^7 (6 real coords + 1 angle)
c3_coords = cy_points_C5[mask].reshape(cy_points_C5.shape[0], -1)
link_points_local = np.concatenate(
    (np.real(c3_coords), np.imag(c3_coords), thetas.reshape(-1, 1)), 
    axis=1
)

print(f"Link points (local R^7 coordinates):")
print(f"  Shape: {link_points_local.shape}")
print(f"  First 6 components: Re(z1), Re(z2), Re(z3), Im(z1), Im(z2), Im(z3)")
print(f"  Last component: theta (angle)")
print(f"  Sample point: {link_points_local[0]}")
print(f"  Theta range: [{thetas.min():.3f}, {thetas.max():.3f}]")

Link points (local R^7 coordinates):
  Shape: (100, 7)
  First 6 components: Re(z1), Re(z2), Re(z3), Im(z1), Im(z2), Im(z3)
  Last component: theta (angle)
  Sample point: [-0.36345359 -0.59842131 -0.35605605  0.24204907 -0.14393444 -0.69199714
  5.18556078]
  Theta range: [0.049, 6.237]


In [20]:
pt_idx = 21
print(data['X_train'][pt_idx])
print(points[pt_idx])
print(cy_points_C5[pt_idx])
print(one_idxs[pt_idx], dropped_idxs[pt_idx])
print(c3_coords[pt_idx])
print(thetas[pt_idx])
print(link_points_local[pt_idx])

[-7.81445993e-01 -1.88755453e-01  1.00000000e+00  2.68457547e-01
 -2.33016858e-01 -7.98111059e-02 -6.32192267e-01 -4.59954260e-17
  3.50870535e-01  8.83778430e-01]
[-7.81445993e-01 -1.88755453e-01  1.00000000e+00  2.68457547e-01
 -2.33016858e-01 -7.98111059e-02 -6.32192267e-01 -4.59954260e-17
  3.50870535e-01  8.83778430e-01]
[-0.78144599-7.98111059e-02j -0.18875545-6.32192267e-01j
  1.        -4.59954260e-17j  0.26845755+3.50870535e-01j
 -0.23301686+8.83778430e-01j]
2 4
[-0.78144599-0.07981111j -0.18875545-0.63219227j  0.26845755+0.35087054j]
5.303511943653816
[-0.78144599 -0.18875545  0.26845755 -0.07981111 -0.63219227  0.35087054
  5.30351194]


## 4. Prepare CY Model

Load the pre-trained CyMetric model for computing the metric.

In [21]:
# Load basis
BASIS = np.load(os.path.join(dirname, 'basis.pickle'), allow_pickle=True)
BASIS_tf = prepare_tf_basis(BASIS)

print(f"Basis loaded:")
print(f"  Type: {type(BASIS)}")

# Build neural network architecture
nn_phi = tf.keras.Sequential()
nn_phi.add(tf.keras.Input(shape=(config['n_in'],)))
for _ in range(config['nlayer']):
    nn_phi.add(tf.keras.layers.Dense(config['nHidden'], activation=config['act']))
nn_phi.add(tf.keras.layers.Dense(config['n_out'], use_bias=False))

print(f"\nNeural network architecture:")
nn_phi.summary()

# Create PhiFSModel
cymetric_model = PhiFSModel(nn_phi, BASIS_tf, alpha=config['alpha'])

# Load pre-trained weights
cymetric_model.nn_phi = tf.keras.models.load_model(cymodel_path)

print(f"\n✅ CyMetric model loaded successfully!")

Basis loaded:
  Type: <class 'dict'>

Neural network architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            64 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,088 (35.50 KB)

 Trainable params: 9,088 (35.50 KB)

 Non-trainable params: 0 (0.00 B)


✅ CyMetric model loaded successfully!


## 5. Compute Geometry

Compute the Hermitian metric, Kähler form, and holomorphic volume form.

### Holomorphic Volume Form

In [ ]:
# Compute holomorphic volume form
holomorphic_volume_form = pg.holomorphic_volume_form(cy_points_C5)

print(f"Holomorphic volume form (Omega):")
print(f"  Shape: {holomorphic_volume_form.shape}")
print(f"  Omega is a complex (3,0)-form on the CY")
print(f"  Sample: {holomorphic_volume_form}")

# Convert to real form
hvf_r, hvf_i = holomorphic_volume_form_to_real(holomorphic_volume_form) ###need to check this function!

print(f"\nReal and imaginary parts:")
print(f"  hvf_r shape: {hvf_r.shape}")
print(f"  hvf_i shape: {hvf_i.shape}")
print(f"  These are real 3-forms on R^6")

Holomorphic volume form (Omega):
  Shape: (100,)
  Omega is a complex (3,0)-form on the CY
  Sample: [ 0.10305573+0.21723129j -0.18385427-0.1055954j  -0.12659825+0.21272109j
 -0.14817739-0.13522107j  0.05230286-0.25103773j  0.22768228-0.01045573j
 -0.29756477+0.15262919j -0.2691326 -0.05324866j  0.14430731+0.18888348j
 -0.17083015+0.11746974j  0.19973444-0.03106225j  0.29312623+0.01812189j
  0.06206269-0.19203071j  0.21175369+0.21191139j -0.16321874+0.12839086j
  0.18933955-0.09910175j  0.23110203-0.05026364j  0.04499285+0.20057541j
  0.14821137-0.15959384j  0.20189324-0.02266839j -0.23252886-0.1317036j
  0.14726073-0.24587766j  0.1121373 -0.30885972j -0.15184816-0.14905434j
 -0.2835573 +0.1512045j   0.01653249+0.20255831j -0.19867126+0.07063361j
 -0.17514759+0.15771262j  0.23003182-0.00452952j -0.15348576+0.17617762j
 -0.14163837-0.19339923j -0.16628312+0.11517989j  0.20158399-0.02349642j
 -0.13587735+0.15532549j -0.16555787-0.13670093j -0.15508181+0.14443624j
 -0.17427078-0.1217451j 

In [ ]:
###testinggg
pt_idx=43
print(holomorphic_volume_form[pt_idx])
print(np.unique(hvf_r[pt_idx]))
print(np.unique(hvf_i[pt_idx]))

(-0.21979149167814588+0.1007256544235424j)
[-0.21979149 -0.10072565  0.          0.10072565  0.21979149]
[-0.21979149 -0.10072565  0.          0.10072565  0.21979149]


### Hermitian Metric and Kähler Form

In [32]:
# Compute Hermitian metric using the CyMetric model
hermitian_metric = cymetric_model(CoordChange_C5R10(cy_points_C5)).numpy()

print(f"Hermitian metric:")
print(f"  Shape: {hermitian_metric.shape}")
print(f"  This is a (1,1) Hermitian tensor on C^3")

# Force hermitian symmetrization to handle GPU numerical precision
hermitian_metric = 0.5 * (hermitian_metric + hermitian_metric.conj().transpose(0, 2, 1))

print(f"  Symmetrized for numerical stability")
print(f"  Sample metric [0, 0, 0]: {hermitian_metric[0, 0, 0]}")

# Convert to Kähler form (real 2-form)
kahler_form_R6 = hermitian_to_kahler_real(hermitian_metric)

print(f"\nKähler form (omega):")
print(f"  Shape in R^6: {kahler_form_R6.shape}")
print(f"  This is a real 2-form on R^6")

# Pad to R^7 for link computations
kahler_form_R7 = np.pad(kahler_form_R6, ((0,0), (0,1), (0,1)), mode='constant')

print(f"  Shape in R^7 (padded): {kahler_form_R7.shape}")
print(f"  Padded with zeros for the theta coordinate")

Hermitian metric:
  Shape: (100, 3, 3)
  This is a (1,1) Hermitian tensor on C^3
  Symmetrized for numerical stability
  Sample metric [0, 0, 0]: (0.09908711165189743+0j)

Kähler form (omega):
  Shape in R^6: (100, 6, 6)
  This is a real 2-form on R^6
  Shape in R^7 (padded): (100, 7, 7)
  Padded with zeros for the theta coordinate


In [56]:
pt_idx=31
print(hermitian_metric[pt_idx])
print()
print(kahler_form_R6[pt_idx])
print()
print(kahler_form_R7[pt_idx])

[[ 0.13901775+0.j         -0.00093934+0.00080865j  0.00476368-0.00201412j]
 [-0.00093934-0.00080865j  0.13845152+0.j         -0.00193253+0.00075699j]
 [ 0.00476368+0.00201412j -0.00193253-0.00075699j  0.13323076+0.j        ]]

[[ 0.          0.          0.          0.          0.00080865 -0.00201412]
 [ 0.          0.          0.         -0.00080865  0.          0.00075699]
 [ 0.          0.          0.          0.00201412 -0.00075699  0.        ]
 [-0.          0.00080865 -0.00201412  0.          0.          0.        ]
 [-0.00080865 -0.          0.00075699  0.          0.          0.        ]
 [ 0.00201412 -0.00075699 -0.          0.          0.          0.        ]]

[[ 0.          0.          0.          0.          0.00080865 -0.00201412  0.        ]
 [ 0.          0.          0.         -0.00080865  0.          0.00075699  0.        ]
 [ 0.          0.          0.          0.00201412 -0.00075699  0.          0.        ]
 [-0.          0.00080865 -0.00201412  0.          0.       

In [ ]:
###test hermiticity
tt=cymetric_model(CoordChange_C5R10(cy_points_C5)).numpy()
pt_idx = 0
tt1 = tt[pt_idx]
print(tt1)
print()
print(tt1.conj().transpose())
print()
diff1 = tt1-tt1.conj().transpose()
print(diff1)
print(f'...max value: {np.max(np.absolute(diff1.real)),np.max(np.absolute(diff1.imag))}')
print()

tt2 = 0.5 * (tt1 + tt1.conj().transpose())
print(tt2)
print()
print(tt2.conj().transpose())
print()
diff2 = tt2-tt2.conj().transpose()
print(diff2)
print(f'...max value: {np.max(np.absolute(diff2.real)),np.max(np.absolute(diff2.imag))}')



[[ 0.09908711+9.3132257e-10j -0.00608823-6.8679694e-03j -0.00433511-8.8626100e-03j]
 [-0.00608823+6.8679694e-03j  0.10332824-4.9476512e-10j -0.01698085+3.2176787e-04j]
 [-0.0043351 +8.8626081e-03j -0.01698085-3.2176753e-04j  0.11197425-9.3132257e-10j]]

[[ 0.09908711-9.3132257e-10j -0.00608823-6.8679694e-03j -0.0043351 -8.8626081e-03j]
 [-0.00608823+6.8679694e-03j  0.10332824+4.9476512e-10j -0.01698085+3.2176753e-04j]
 [-0.00433511+8.8626100e-03j -0.01698085-3.2176787e-04j  0.11197425+9.3132257e-10j]]

[[ 0.0000000e+00+1.8626451e-09j  4.6566129e-10+0.0000000e+00j -4.6566129e-10-1.8626451e-09j]
 [-4.6566129e-10+0.0000000e+00j  0.0000000e+00-9.8953024e-10j -1.8626451e-09+3.4924597e-10j]
 [ 4.6566129e-10-1.8626451e-09j  1.8626451e-09+3.4924597e-10j  0.0000000e+00-1.8626451e-09j]]
...max value: (1.8626451e-09, 1.8626451e-09)

[[ 0.09908711+0.j         -0.00608823-0.00686797j -0.00433511-0.00886261j]
 [-0.00608823+0.00686797j  0.10332824+0.j         -0.01698085+0.00032177j]
 [-0.00433511+0.

## 6. Compute G2 Forms

Compute the G2 3-form using the wedge product.

In [ ]:
# Create dtheta (differential of the angle coordinate)
dthetas = np.concatenate(
    (np.zeros((cy_points_C5.shape[0], 6)), np.ones((cy_points_C5.shape[0], 1))), 
    axis=1
)

print(f"dtheta (1-form):")
print(f"  Shape: {dthetas.shape}")
print(f"  Values: [0, 0, 0, 0, 0, 0, 1] (only non-zero in theta direction)")

# Compute omega ^ dtheta
print(f"\nComputing omega ^ dtheta...")
omega_wedge_dtheta = np.array([
    wedge(kahler_form_R7[i], dthetas[i]) 
    for i in range(cy_points_C5.shape[0])
])

print(f"  omega ^ dtheta shape: {omega_wedge_dtheta.shape}")
print(f"  This is a 3-form on R^7")

dtheta (1-form):
  Shape: (100, 7)
  Values: [0, 0, 0, 0, 0, 0, 1] (only non-zero in theta direction)

Computing omega ^ dtheta...
  omega ^ dtheta shape: (100, 7, 7, 7)
  This is a 3-form on R^7


In [67]:
pt_idx=0
print(kahler_form_R7[pt_idx])
print(omega_wedge_dtheta[pt_idx])


[[ 0.          0.          0.          0.         -0.00686797 -0.00886261  0.        ]
 [ 0.          0.          0.          0.00686797  0.          0.00032177  0.        ]
 [ 0.          0.          0.          0.00886261 -0.00032177  0.          0.        ]
 [-0.         -0.00686797 -0.00886261  0.          0.          0.          0.        ]
 [ 0.00686797 -0.          0.00032177  0.          0.          0.          0.        ]
 [ 0.00886261 -0.00032177 -0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.        ]]
[[[ 0.          0.          0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.        

In [68]:
# Assemble the G2 3-form: phi = omega ^ dtheta + Re(Omega)
g2form_R7 = omega_wedge_dtheta.copy()
g2form_R7[:, :6, :6, :6] += hvf_r

print(f"G2 3-form (phi):")
print(f"  Shape: {g2form_R7.shape}")
print(f"  phi = omega ^ dtheta + Re(Omega)")
print(f"  This defines the G2 structure on the link R^7")

# Check non-zero components
non_zero_mask = np.abs(g2form_R7) > 1e-10
n_nonzero = np.sum(non_zero_mask)
total_components = np.prod(g2form_R7.shape)

print(f"\nSparsity analysis:")
print(f"  Non-zero components: {n_nonzero} / {total_components}")
print(f"  Sparsity: {(1 - n_nonzero/total_components)*100:.1f}%")

# Statistics
print(f"\nStatistics:")
print(f"  Min: {np.min(g2form_R7):.6f}")
print(f"  Max: {np.max(g2form_R7):.6f}")
print(f"  Mean: {np.mean(g2form_R7):.6f}")
print(f"  Std: {np.std(g2form_R7):.6f}")

G2 3-form (phi):
  Shape: (100, 7, 7, 7)
  phi = omega ^ dtheta + Re(Omega)
  This defines the G2 structure on the link R^7

Sparsity analysis:
  Non-zero components: 8400 / 34300
  Sparsity: 75.5%

Statistics:
  Min: -0.308860
  Max: 0.308860
  Mean: 0.000036
  Std: 0.065202


## 7. Optional: Compute G2 Metric

Compute the G2 metric from the 3-form (if needed).

In [69]:
# Compute G2 metric
print("Computing G2 metric from 3-form...")
print("(This may take a moment)")

g2_metric = np.array([compute_gG2(g2) for g2 in g2form_R7])

print(f"\nG2 metric:")
print(f"  Shape: {g2_metric.shape}")
print(f"  This is a Riemannian metric on R^7")
print(f"  Sample metric [0, 0, 0]: {g2_metric[0, 0, 0]:.6f}")

# Check symmetry
is_symmetric = np.allclose(g2_metric, g2_metric.transpose(0, 2, 1))
print(f"  Metric is symmetric: {is_symmetric}")

# Check positive definiteness (eigenvalues)
eigenvalues = np.linalg.eigvalsh(g2_metric)
min_eigenvalue = np.min(eigenvalues)
print(f"  Minimum eigenvalue: {min_eigenvalue:.6f} (should be > 0)")
print(f"  Positive definite: {min_eigenvalue > 0}")

Computing G2 metric from 3-form...
(This may take a moment)


/Users/ed/Documents/Projects/G2/Flows/G2Metric/github/geometry/geometry.py:139: RuntimeWarning: invalid value encountered in scalar power
  factor = (1 / pow(36, 1 / 9)) * (1 / pow(detB, 1 / 9))


KeyboardInterrupt: 

## 8. Vectorized Forms for Neural Network

Convert forms to vector representation for use with neural networks.

In [ ]:
# Convert 3-forms to vectors
g2form_vecs = form_to_vec(tf.convert_to_tensor(g2form_R7))

print(f"Vectorized G2 3-form:")
print(f"  Original shape: {g2form_R7.shape}")
print(f"  Vector shape: {g2form_vecs.shape}")
print(f"  Number of independent components: {g2form_vecs.shape[1]}")

# Analyze component statistics
component_means = np.mean(g2form_vecs, axis=0)
component_stds = np.std(g2form_vecs, axis=0)
component_abs_means = np.mean(np.abs(g2form_vecs), axis=0)

# Find zero components
zero_threshold = 1e-10
zero_components = np.where(component_abs_means < zero_threshold)[0]
nonzero_components = np.where(component_abs_means >= zero_threshold)[0]

print(f"\nComponent analysis:")
print(f"  Zero components: {len(zero_components)} / {len(component_means)}")
print(f"  Zero component indices: {zero_components.tolist()}")
print(f"  Non-zero components: {len(nonzero_components)}")
print(f"  Non-zero indices: {nonzero_components.tolist()}")

print(f"\nNon-zero component statistics:")
if len(nonzero_components) > 0:
    print(f"  Mean of abs values: {component_abs_means[nonzero_components][:5]}... (first 5)")
    print(f"  Std dev: {component_stds[nonzero_components][:5]}... (first 5)")

## 9. Test Summary

Summary of all computed objects for verification.

In [ ]:
print("="*70)
print("TESTING SUMMARY")
print("="*70)

print(f"\n✅ Successfully computed all LinkSample components:")
print(f"\n1. Configuration:")
print(f"   - Loaded from: {config_path}")
print(f"   - Ambient space: CP^{config['ambient']-1}")

print(f"\n2. Points Generated:")
print(f"   - Number of points: {n_pts}")
print(f"   - CY points (C^5): {cy_points_C5.shape}")
print(f"   - Link points (R^7): {link_points_local.shape}")
print(f"   - Number of patches: {len(unique_patches)}")

print(f"\n3. CyMetric Model:")
print(f"   - Loaded from: {cymodel_path}")
print(f"   - Architecture: {config['nlayer']} layers, {config['nHidden']} hidden units")

print(f"\n4. Geometric Objects:")
print(f"   - Holomorphic volume form: {holomorphic_volume_form.shape}")
print(f"   - Hermitian metric: {hermitian_metric.shape}")
print(f"   - Kähler form (R^7): {kahler_form_R7.shape}")
print(f"   - G2 3-form: {g2form_R7.shape}")
if 'g2_metric' in locals():
    print(f"   - G2 metric: {g2_metric.shape}")

print(f"\n5. Vectorized Data:")
print(f"   - G2 form vectors: {g2form_vecs.shape}")
print(f"   - Non-zero components: {len(nonzero_components)} / {g2form_vecs.shape[1]}")

print(f"\n✅ All components tested successfully!")
print("="*70)